Helper Functions

In [1]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date
from zipfile import ZipFile
import hashlib
import os
from uuid import uuid1

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
   
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
   
def download_pdf(url,path,filename):
    try:
       
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
   
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

   
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

def download_zip(url):
    # Downloading the file by sending the request to the URL
    req = requests.get(url)
    # Split URL to get the file name
    filename = url.split('/')[-1]
    # Writing the file to the local file system
    with open(filename,'wb') as output_file:
        output_file.write(req.content)
    print('Downloading Completed')

    
def extract_zip (name): 
    # specifying the zip file name  
    file_name = name    
    try:
    # opening the zip file in READ mode
        with ZipFile(file_name, 'r') as zip:
            # printing all the contents of the zip file
            zip.printdir()
        
            # extracting all the files
            print('Extracting all the files now...')
            zip.extractall()
            print('Done!')
    except:
        return "error"
def scan_dir(dir_name):
    if dir_name == "Transfer – Forum NC Orders":
        dir_name = "Transfer - Forum NC Orders"
    out_list = []
    print("---------------------{}----------------------".format(dir_name))
    for filename in os.scandir(dir_name):
        if filename.is_file():
            # print(filename.path.split("/")[-1].split(".")[0])
            title = filename.path.split("\\")[-1]
            file_name = title
            out_list.append(title)
    return out_list

1) 

In [2]:
BASE = "https://georgiacourts.gov"
start = "https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/"


soup = connect_to_page(start)
cat_refs = []
cat_names = []


for i in soup.find_all("h4", class_ = "vc_tta-panel-title vc_tta-controls-icon-position-left"):
    a = i.find("a", href = True)
    cat_refs.append(start+a['href'])
    cat_names.append(a.text)
print(cat_refs)
print(cat_names)


['https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/#1566231380414-5e189c6c-bce0', 'https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/#1566244668513-d643f872-93bf', 'https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/#1566244969529-97e234f4-7038', 'https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/#1566241462728-b588212e-e7e2', 'https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/#1566242970836-b2a5117a-f6bb']
['Civil Forms', 'Juror Issues', 'Recusal Orders', 'Criminal Forms', 'Other Criminal Items']


2.

In [ ]:
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","type"])
form_dest = "All_forms/"
jur = 'GA'


soup = connect_to_page(cat_refs[0])
wrapper = soup.find_all("div", class_ = "vc_tta-panel-body")
group_num = 0

for cat in wrapper:
    group_outer = cat_names[group_num]
    group_num+=1
    print("-------------{}---------------------".format(group_outer))
    links_per_cat =  cat.find_all("a", href=True)

    for ref in links_per_cat:
        if ".zip" in ref['href']:
            url = BASE+ref['href']
            zip_name = url.split("/")[-1]
            folder_name = re.sub(r"\([^()]*\)","",ref.text)
            folder_name = folder_name.strip()
            # print(name)
            zip = download_zip(url)
            print(zip_name)
            status = extract_zip(zip_name)
            fileid = hashme(url)
            group_inner = zip_name.split(".")[0]
            if status!="error":
                # print("Transfer - Forum NC Orders" == "Transfer – Forum NC Orders")
                files_in_zip = scan_dir(folder_name)
                for file in files_in_zip:
                    print(file, " IN FOR LOOP")
                    if "doc" in file.split(".")[1]:
                        
                        file_type =  "doc"
                    else:
                        file_type = "pdf"
                    # fileid = uuid1().hex
                    files_df = files_df.append(pd.DataFrame([[fileid,jur,start,file,group_inner,url,file,today, file_type]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "type"]))
        
            else:
                print("---------------")
                print("ERROR: ", zip_name , group_inner)
                print("---------------")

            
        elif ".doc" in ref['href'] and "http" not in ref['href']:
            url = BASE + ref['href']
            fileid = hashme(url)
            filename = url.split("/")[-1].split(".")[0]
            file_type = "doc"
            title = ref.text
            download_docx(url, fileid+'.{}'.format(file_type))
            files_df = files_df.append(pd.DataFrame([[fileid,jur,start,title,group_outer,url,filename,today, file_type]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "type"]))

            print(url)

        elif ".doc" in ref['href']: #case where "http" part of string
            url = ref['href']
            fileid = hashme(url)
            filename = url.split("/")[-1].split(".")[0]
            file_type = "doc"
            title = ref.text
            download_docx(url, fileid+'.{}'.format(file_type))
            files_df = files_df.append(pd.DataFrame([[fileid,jur,start,title,group_outer,url,filename,today, file_type]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "type"]))
            print(ref.text)
            print(url)

        else: #case where its .pdf
            url = BASE + ref['href']
            fileid = hashme(url)
            filename = url.split("/")[-1].split(".")[0]
            file_type = "pdf"
            title = ref.text
            download_pdf(url, form_dest,fileid+'.{}'.format(file_type))
            files_df = files_df.append(pd.DataFrame([[fileid,jur,start,title,group_outer,url,filename,today, file_type]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "type"]))

            print(ref.text)
            print(url)

files_df.to_csv("GA-V1.csv",index=False, encoding="utf-8")

    

GA was the first state I encountered with compressed folders containing multiple forms (i.e. zip files). I downloaded them and extracted their content. They were assigned a unique id, which was shared among all the forms inside the zip file. Another issue with files inside the zip was that a filename could not be extracted (title is the same as the filename). 

1. Extracts the different categories and the links to those categories
2. Extracts the individual forms and zip files from the categories. Due to the presence of zip files, some additional functions are used (look helper functions)

Helper functions are essential functions used in every states code